In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


def getTime(data):
    dff = data.copy()
    ## теперь нужно разобраться со временем
#     df['Time'] = pd.Series(str(df['pass_dttm'].dt.hour) + ':' + str(((df['pass_dttm'].dt.minute // 30)*30))  + ' - '+ str(df['pass_dttm'].dt.hour+1) + ':' + str(30 - ((df['pass_dttm'].dt.minute // 30)*30)) ,index = df.index)
    j = 0 
    dff['Time'] = pd.Series(0,index = dff.index)
    for i in dff['pass_dttm']:
#         val = int((i.minute // 30)*3)
#         val1 = i.hour
#         val1 = str(i.hour) + ':' + str(val) + '0 - ' + str(i.hour + 1*int(val/3)) + ':' + str(3 - val) + '0'
        dff['Time'][j] = str(i.hour) + ':' + str(i.minute // 30)
        j += 1
    return dff

def getTime2(data):
    dff = data.copy()
    dff['hour'] = dff['pass_dttm'].dt.hour
    dff['min'] = dff['pass_dttm'].dt.minute//30
    dff['hour'] = pd.Series(dff['hour'], dtype='string')
    dff['min'] = pd.Series(dff['min'], dtype='string')
    dff['Time'] = dff['hour']+'_'+dff['min']
    dff.drop(['min','hour'],axis=1,inplace=True)
    return dff

## функция заменяем одни и те же названия
def replaceOnes(data):
    val1 = 'СК учащегося 30 дней'
    val11 = '30 дней СК учащегося'
    val2 = 'СК аспиранта 90 дней'
    val22 = '90 дней СК аспиранта'
    val4 = 'СК аспиранта 30 дней'
    val44 = '30 дней СК аспиранта'
    val3 = 'СК учащегося 90 дней'
    val33 = '90 дней СК учащегося'
    val5 = 'СК студента 30 дней'
    val55 = '30 дней СК студента'
    val6 = 'СК студента 90 дней'
    val66 = '90 дней СК студента'
    val7 = 'СК ординатора 30 дней'
    val77 = '30 дней СК ординатора'
    val8 = 'СК ординатора 90 дней'
    val88 = '90 дней СК ординатора'
    val9 = 'СК ассистента-стажера 30 дней'
    val99 = '30 дней СК ассистента-стажера'
    M = [val1,val2,val3,val4,val5,val6,val7,val8,val9]
    M1 = [val11,val22,val33,val44,val55,val66,val77,val88,val99]

    df2 = data.copy()
    for i in range(len(df2)):
        v = df2.ticket_type_nm[i]
        if v in M:
            ind = M.index(v)
            ## тогда заменяем
            df2.ticket_type_nm.iloc[i] = M1[ind]
    return df2

def OneHot(data):
    df3 = data.copy()
    df4 = df3[['ticket_type_nm', 'Time']]
    ohe = OneHotEncoder(sparse=False)
    ohe.fit(df4)
    ohe_model = ohe.transform(df4)
    df3[ohe.get_feature_names()] = ohe_model
    ## теперь удаляем ticket_type_nm, Time и station_id
    df3.drop(['ticket_type_nm', 'Time'], axis=1, inplace=True)
    return df3

def linRegr(df1,df2):
    X_train, X_test, y_train, y_test = train_test_split(df1, df2, test_size=0.33, random_state=42)
    reg = LinearRegression().fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    return mape(y_pred, y_test)

df = pd.read_csv('data/train_dataset_train.csv', sep=',')

### пишем функцию, которая приводит датафрейм к нормальному виду для обучения
## выкинем лишние столбцы
df.drop(['id','ticket_id','entrance_id','station_nm','entrance_nm','line_nm','line_id'], axis=1, inplace = True)
## переводим столбец pass_dttm в datetime
df['pass_dttm'] = pd.to_datetime(df['pass_dttm'])
## получаем столбец Time - срез по 30 минут
df = getTime2(df)
## заменяем одни и те же названия
df = replaceOnes(df)

## переводим в категориальные данные столбцы Time, ticket type nm, station id
df = OneHot(df)
df0 = df['time_to_under']
df.drop(['time_to_under','label','pass_dttm'],axis=1,inplace=True)
# df.head()
## создадим второй датафрейм, без столбца label
## предсказание time to under
# mapeResult = linRegr(df,df0)
# return mapeResult

# if __name__ == '__main__':
#     df_main = pd.read_csv('data/train_dataset_train.csv', sep=',')
#     prepare(df_main)


C:\Users\ProDell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\ProDell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [75]:
df2 = pd.read_csv('data/train_dataset_train.csv', sep=',')
df['line_id'] = df2['line_id']
# df['station_id'] = pd.Series(df['station_id'],dtype = 'category')
del(df2)

In [76]:
df.head()

,station_id,x0_1 сутки ЕДИНЫЙ ТК,x0_1 сутки Пригород ТК,x0_3 суток Единый ТК,x0_30 дней,x0_30 дней Пригород,x0_30 дней ЕДИНЫЙ ТК,x0_30 дней Пригород ТК,x0_30 дней СК аспиранта,x0_30 дней СК ассистента-стажера,...,x1_5_1,x1_6_0,x1_6_1,x1_7_0,x1_7_1,x1_8_0,x1_8_1,x1_9_0,x1_9_1,line_id
0,11007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11
1,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,2006,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [77]:
from sklearn.metrics import r2_score
def linRegr(df1,df2):
    X_train, X_test, y_train, y_test = train_test_split(df1, df2, test_size=0.3, random_state=42)
    reg = LinearRegression().fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2

In [78]:
df['station_id'] = pd.Series(df['station_id'], dtype='category')
Result = linRegr(df,df0)

In [79]:
Result

0.5133799764754743

In [16]:
from sklearn import linear_model
def LassoReg(df1,df2):
    X_train, X_test, y_train, y_test = train_test_split(df1, df2, test_size=0.3, random_state=42)
    reg = linear_model.Lasso(alpha=0.05)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2
LassoReg(df,df0)

0.5107836167845914

In [21]:
###
from sklearn.tree import DecisionTreeRegressor
def DecisionTree(df1,df2):
    X_train, X_test, y_train, y_test = train_test_split(df1, df2, test_size=0.3, random_state=42)
    reg = DecisionTreeRegressor()
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    return r2
DecisionTree(df,df0)

0.4865640965507819

In [1]:
### теперь попробуем решить задачу классификации на тех же данных
from sklearn.metrics import recall_score

In [22]:
df2 = pd.read_csv('data/train_dataset_train.csv', sep=',')
dfPRED = df2['label']
dfPRED = pd.Series(dfPRED,dtype='category')
del(df2)
dfPRED.head()
df.head()

,station_id,x0_1 сутки ЕДИНЫЙ ТК,x0_1 сутки Пригород ТК,x0_3 суток Единый ТК,x0_30 дней,x0_30 дней Пригород,x0_30 дней ЕДИНЫЙ ТК,x0_30 дней Пригород ТК,x0_30 дней СК аспиранта,x0_30 дней СК ассистента-стажера,...,x1_5_0,x1_5_1,x1_6_0,x1_6_1,x1_7_0,x1_7_1,x1_8_0,x1_8_1,x1_9_0,x1_9_1
0,Лефортово,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Войковская,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Войковская,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Войковская,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Войковская,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df, dfPRED, test_size=0.3)
# X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.3)
gbm = lgb.LGBMClassifier(objective='multiclass', num_leaves=5, n_estimators=10)
gbm.fit(X_train, y_train)
y_predict = gbm.predict(X_test)

In [11]:
recall = recall_score(y_test, y_predict, average='micro')
print(recall)

0.0595251552823496


In [17]:
## попробуем logisticregres
from sklearn.linear_model import LogisticRegression

In [54]:
def LogReg(df1,df2):
    X_train, X_test, y_train, y_test = train_test_split(df1, df2, test_size=0.3)
    # X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.3)
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_predict = logreg.predict(X_test)
    recall = recall_score(y_test, y_predict, average='micro')
    return recall
LogReg(df,dfPRED)

C:\Users\ProDell\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.023455043735697678

In [70]:
df2 = pd.read_csv('data/train_dataset_train.csv', sep=',')
df['station_id'] = df2['station_id']
df['station_id'] = pd.Series(df['station_id'],dtype = 'category')
del(df2)

In [71]:
df.head()

,station_id,x0_1 сутки ЕДИНЫЙ ТК,x0_1 сутки Пригород ТК,x0_3 суток Единый ТК,x0_30 дней,x0_30 дней Пригород,x0_30 дней ЕДИНЫЙ ТК,x0_30 дней Пригород ТК,x0_30 дней СК аспиранта,x0_30 дней СК ассистента-стажера,...,x1_5_0,x1_5_1,x1_6_0,x1_6_1,x1_7_0,x1_7_1,x1_8_0,x1_8_1,x1_9_0,x1_9_1
0,11007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2006,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
from sklearn.tree import DecisionTreeClassifier
def DecTree(df1,df2):
    X_train, X_test, y_train, y_test = train_test_split(df1, df2, test_size=0.3)
    # X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.3)
    regressor = DecisionTreeClassifier()
    regressor.fit(X_train, y_train)
    y_predict = regressor.predict(X_test)
    recall = recall_score(y_test, y_predict, average='micro')
    return recall
res2 = DecTree(df,dfPRED)

In [73]:
res2

0.10532008175810477

In [80]:
final = 0.5*Result + 0.5*res2
final

0.30935002911678955